# Retrieve a OneView authentication token for a managed iLO 5

Version 0.14

## Introduction 

This notebook uses the [HPE OneView Ansible Galaxy collection](https://galaxy.ansible.com/hpe/oneview) to connect to a OneView appliance, get a server that is managed by this appliance and get a security token for that server's iLO. This token will be used in subsquent sections of the lab to connect to the iLO without authenticating again, using the Single Sign On (SSO) feature of OneView and iLO.

OK, for the sake of transparency, the unabated truth for this lab is that you will be using simulated hardware. You are going to connect to a OneView appliance that contains code that simulates responses from managed hardware. While these simulated responses do allow getting an iLO security token, they do not fully simulate an iLO. So in subsequent sections of this lab, you will be using another simulator that fully implements the Redfish API like an iLO. The 2 simulators are independent of each other, in other words the token you are going to obtain in this section cannot really be used in the following sections. You will just have to trust us that it really works this way with real hardware. Yes, we've tested it !

## <a id = "EnvironmentPreparation">Environment preparation</a>

In [ ]:
########## Environment preparation ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=1

# OneView variables
OneViewSimulatorIP={{ OVVMSIMU1IP }}

cat > ${NbId}/config.json << __EoF__
{
    "ip": "${OneViewSimulatorIP}",
    "credentials": {
      "userName": "Administrator",
      "authLoginDomain": "",
      "password": "password"
    },
    "api_version": 1800
  }
__EoF__

echo "You are ready to continue...."


First we need to install the [HPE OneView Ansible Galaxy collection](https://galaxy.ansible.com/hpe/oneview). This is simply done with the following command:

In [ ]:
/opt/jupyterhub/bin/ansible-galaxy collection install hpe.oneview

The collection is installed (by default) in ~/.ansible/collections. We also need to look at the requirements for this collection, as they are not installed by the ansible-galaxy collection install command.

In [ ]:
cat ~/.ansible/collections/ansible_collections/hpe/oneview/requirements.txt

These are Python packages to be installed with pip. You can safely ignore hpICsp which is no longer used. ansible is kind of obvious and is already installed. hpeOneView is the [Python OneView SDK](https://github.com/HewlettPackard/oneview-python) which has aleady been installed in your environment.

We are now ready to run a playbook to get a security token for an iLO of a managed server. Before running it, let's look at it, open [the playbook](1/GetiLOSSOToken.yml), it will open in a new tab, come back to this tab for explanations. The playbook starts with these lines:

```
- hosts: localhost
  gather_facts: false
  collections:
    - hpe.oneview
```

The playbook always runs on localhost. This is not the traditional ansible way where ansible will connect to various hosts to perform its tasks. HPE OneView is an appliance, you cannot ssh into it, so you cannot have a OneView appliance in an ansible hosts inventory, this would not work. Instead, the ansible OneView collection runs locally but connects to the REST API of a OneView appliance specified in a config file we will look at in a minute.

We are not particularly interested in details of the host we are running the playbook on, so `gather_facts: false` allows to speed up the execution a bit.

And finally we declare the collection we are going to use in this playbook.

Next is our first task:
```
  - name: Get server
    oneview_server_hardware_facts:
      config: "config.json"  
      name: "0000A66101, bay 7"
      options:
      - remoteConsoleUrl: true

```

`oneview_server_hardware_facts` is the role we want to use from our Ansible Galaxy collection. It essentially translates to a REST API call to OneView with a GET /rest/server-hardware

`config: "config.json"` is where we define which OneView appliance we want to connect to. You can open [this JSON file](1/config.json), it simply contains the IP address and credentials of a OneView appliance (there are ways to avoid having a password in clear text like this, but they are beyond the scope of this lab), and the API version we want to use. Each version of OneView implements a version of the REST API (and a number of previous versions for backwards compatibility). See [the About page of the OneView REST API Reference](https://techlibrary.hpe.com/docs/enterprise/servers/oneview5.5/cicf-api/en/index.html#about)

`name: "0000A66101, bay 7"` is the name of the server we want to access.

`      options:
      - remoteConsoleUrl: true
` says that we want to retrieve in our GET call the remoteConsoleUrl attribute which contains the iLO token we want

Now we want to parse the results of our REST API call to extract just the bit we are interested in, it is in the session key parameter in the remoteConsoleUrl, and we store it in the `iloToken` variable:
```
  - set_fact:
      iloToken: "{% raw %}{{ server_hardware_remote_console_url.remoteConsoleUrl | regex_search(regexp, '\\1') }}{% endraw %}"
    vars:
      regexp: 'sessionkey=([0-9a-f]+)'
```

And finally we print the result:
```
  - name: print result
    debug: 
      var: iloToken
```

Now you can run the playbook:

In [ ]:
/opt/jupyterhub/bin/ansible-playbook 1/GetiLOSSOToken.yml

You can now move to [the next lab](2-RedfishAnsibleUsingBuiltinUri.ipynb) and start talking Redfish to an iLO.